# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [3]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,susuman,62.7833,148.1667,11.88,78,100,1.56,RU,1723097075
1,1,jamestown,42.0970,-79.2353,19.45,94,75,2.57,US,1723097076
2,2,georgetown,5.4112,100.3354,32.96,62,20,6.69,MY,1723096821
3,3,tura,25.5198,90.2201,29.28,74,58,3.72,IN,1723097078
4,4,thanh hoa,19.8000,105.7667,36.82,48,0,1.89,VN,1723097079


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [6]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha=0.5
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [8]:
# Narrow down cities that fit criteria and drop any results with null values
#A max temperature lower than 27 degrees but higher than 21
#Wind speed less than 4.5 m/s
#Zero cloudiness
ideal_city_conditions_df = city_data_df[
    (city_data_df['Max Temp'] < 27) &
    (city_data_df['Max Temp'] > 21) &
    (city_data_df['Wind Speed'] < 4.5) &
    (city_data_df['Cloudiness'] == 0) 
]
# Drop any rows with null values
ideal_cities_df = ideal_city_conditions_df.dropna()

# Display sample data
ideal_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
11,11,hamilton,39.1834,-84.5333,21.90,88,0,3.09,US,1723097088
39,39,toliara,-23.3500,43.6667,23.85,51,0,2.13,MG,1723097122
40,40,ribeira grande,38.5167,-28.7000,22.10,84,0,4.02,PT,1723097123
57,57,remire-montjoly,4.9167,-52.2667,25.02,94,0,0.00,GF,1723097143
131,131,ulaangom,49.9811,92.0667,26.52,28,0,3.69,MN,1723097233
188,188,praia da vitoria,38.7333,-27.0667,21.53,85,0,1.92,PT,1723097303
225,225,vila velha,-20.3297,-40.2925,21.08,87,0,3.13,BR,1723097349
228,228,eyguieres,43.6948,5.0313,24.51,73,0,1.03,FR,1723097353
231,231,horta,38.5333,-28.6333,21.68,84,0,3.13,PT,1723097356
253,253,tete,-16.1564,33.5867,21.05,60,0,1.54,MZ,1723097384


### Step 3: Create a new DataFrame called `hotel_df`.

In [10]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[['City','Country','Lat','Lng','Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
11,hamilton,US,39.1834,-84.5333,88,
39,toliara,MG,-23.3500,43.6667,51,
40,ribeira grande,PT,38.5167,-28.7000,84,
57,remire-montjoly,GF,4.9167,-52.2667,94,
131,ulaangom,MN,49.9811,92.0667,28,
188,praia da vitoria,PT,38.7333,-27.0667,85,
225,vila velha,BR,-20.3297,-40.2925,87,
228,eyguieres,FR,43.6948,5.0313,73,
231,horta,PT,38.5333,-28.6333,84,
253,tete,MZ,-16.1564,33.5867,60,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [ ]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'categories':'accommodation.hotel',
    'apiKey': geoapify_key,
    'limit': 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lng},{lat},{radius}'
    params["bias"] = f'proximity:{lng},{lat}'

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.get(base_url,params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
hamilton - nearest hotel: North Vista Manor
toliara - nearest hotel: Ambary
ribeira grande - nearest hotel: Hospedaria JSF
remire-montjoly - nearest hotel: Complexe Belova
ulaangom - nearest hotel: АНДУУД
praia da vitoria - nearest hotel: Salles
vila velha - nearest hotel: Hotel Prainha
eyguieres - nearest hotel: Domaine de Roquerousse


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

hotel_map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    hover_cols='all',
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "Country",
    alpha = .5
)

# Display the map
hotel_map_plot